<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import numpy as np
import gensim
import os
import re
import sys

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [2]:
df  = pd.read_csv('./data/amazon.csv')

In [4]:
titles = df['name']

In [7]:
STOPWORDS = set(STOPWORDS).union(set(['amazon', 'amazon.com']))

In [8]:
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]


In [9]:
corpus = df['reviews.text'].apply(tokenize).values

In [10]:
corpus[1]

['bulk', 'expensive', 'way', 'products', 'like']

In [11]:
id2word = corpora.Dictionary(corpus)

In [12]:
id2word.doc2bow(tokenize('This is my example reveiw of my batteries. They were not the best but buying in bulk was great'))

[(10, 1), (20, 1), (63, 1), (232, 1), (461, 1)]

In [13]:
sys.getsizeof(id2word)

56

In [14]:
len(id2word.keys())

9619

In [15]:
id2word.filter_extremes(no_below=10, no_above=0.75)
len(id2word.keys())

2356

In [16]:
corpum = [id2word.doc2bow(text) for text in corpus]

In [ ]:
lda = LdaMulticore(corpus=corpum,
                  id2word=id2word,
                  random_state=42,
                  num_topics=5,
                  passes=50,
                  workers=4,
                  )

In [17]:
words = [re.findall(r'"([^"]*)"', topic[1]) for topic in lda.print_topics()]
                    

In [19]:
topics = [' '.join(topic[0:5]) for topic in words]

In [20]:
print(topics[0])

tablet great loves old kids


In [21]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

/Users/lambda_school_loaner_153/anaconda3/envs/unit4/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [22]:
pyLDAvis.gensim.prepare(lda, corpum, id2word)

/Users/lambda_school_loaner_153/anaconda3/envs/unit4/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.237253 -0.147327       1        1  24.260595
0     -0.218591 -0.096870       2        1  20.496420
4     -0.078771  0.044313       3        1  20.363995
1     -0.061396  0.038367       4        1  18.237314
2      0.121505  0.161517       5        1  16.641672, topic_info=     Category         Freq       Term        Total  loglift  logprob
15    Default  5471.000000      price  5471.000000  30.0000  30.0000
1829  Default  6973.000000     tablet  6973.000000  29.0000  29.0000
1625  Default  2975.000000     kindle  2975.000000  28.0000  28.0000
1494  Default  2168.000000      loves  2168.000000  27.0000  27.0000
479   Default  2115.000000        old  2115.000000  26.0000  26.0000
18    Default  9529.000000      great  9529.000000  25.0000  25.0000
31    Default  5996.000000       good  5996.000000  24.0000  24.0000
183   Default  2432.000000       kids  2432.000000  23.0000  23.0000
1631  Default  1523.000000      books  1523.000000  22.0000  22.0000
677   Default  2928.000000       easy  2928.000000  21.0000  21.0000
99    Default  1747.000000       year  1747.000000  20.0000  20.0000
7     Default  2177.000000       work  2177.000000  19.0000  19.0000
20    Default  2246.000000       long  2246.000000  18.0000  18.0000
21    Default  3812.000000     amazon  3812.000000  17.0000  17.0000
704   Default  1210.000000       read  1210.000000  16.0000  16.0000
592   Default  1721.000000      games  1721.000000  15.0000  15.0000
1284  Default  1852.000000       apps  1852.000000  14.0000  14.0000
830   Default  1203.000000    reading  1203.000000  13.0000  13.0000
358   Default  1116.000000      value  1116.000000  12.0000  12.0000
1368  Default  1632.000000     screen  1632.000000  11.0000  11.0000
84    Default  3417.000000       love  3417.000000  10.0000  10.0000
1084  Default   941.000000   daughter   941.000000   9.0000   9.0000
17    Default  1105.000000      brand  1105.000000   8.0000   8.0000
121   Default   976.000000       size   976.000000   7.0000   7.0000
5     Default  1486.000000    quality  1486.000000   6.0000   6.0000
25    Default  3189.000000     bought  3189.000000   5.0000   5.0000
654   Default   945.000000        son   945.000000   4.0000   4.0000
783   Default  1058.000000       gift  1058.000000   3.0000   3.0000
71    Default   749.000000     brands   749.000000   2.0000   2.0000
398   Default   891.000000       deal   891.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
655    Topic5   368.205994      sound   441.356445   1.6121  -4.8717
1244   Topic5   121.463417      adult   132.431183   1.7068  -5.9807
1424   Topic5    98.835579      micro   105.821213   1.7250  -6.1869
989    Topic5    89.095009    excited    94.818878   1.7310  -6.2906
1025   Topic5   130.411621  bluetooth   152.402802   1.6374  -5.9096
441    Topic5   202.626785      tried   264.876862   1.5254  -5.4690
816    Topic5   251.543655      alexa   361.886475   1.4295  -5.2527
128    Topic5   194.445190     lasted   259.252777   1.5056  -5.5102
14     Topic5   481.378174      happy   871.090332   1.2002  -4.6037
108    Topic5   286.746002       know   444.444824   1.3550  -5.1217
136    Topic5   261.588837    working   392.830780   1.3867  -5.2135
132    Topic5   185.420547     remote   255.465363   1.4728  -5.5577
714    Topic5   163.446259    service   212.836319   1.5292  -5.6838
37     Topic5   456.852661   purchase   996.001953   1.0139  -4.6560
211    Topic5   586.681152       best  1520.036621   0.8413  -4.4058
26     Topic5   660.981934        buy  2343.534180   0.5276  -4.2866
223    Topic5   253.431503        day   469.824310   1.1760  -5.2452
25     Topic5   680.242310     bought  3189.670410   0.2480  -4.2579
159    Topic5   327.421692         ve   868.977600   0.8172  -4.9891
21     Topic5   630.470032     amazon  3812.232666  -0.0062 

In [23]:
lda[corpum[0]]

[(0, 0.022283321),
 (1, 0.25287986),
 (2, 0.022840405),
 (3, 0.67972517),
 (4, 0.022271248)]

In [25]:
distro = [lda[d] for d in corpum]

In [26]:
distro[0]

[(0, 0.022283332),
 (1, 0.2529442),
 (2, 0.022839917),
 (3, 0.67966133),
 (4, 0.022271251)]

In [27]:
def update(doc):
    d_dist = {k: 0 for k in range(0, 15)}
    for t in doc:
        d_dist[t[0]] = t[1]
    return d_dist

In [28]:
new_distro = [update(d) for d in distro]

In [29]:
d_dist = {k: 0 for k in range(15)}
d_dist

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0}

['tablet great loves old kids',
 'tablet amazon apps app good',
 'bought buy amazon best charge',
 'great price good long work',
 'kindle books use easy great']

[{0: 0.022283332,
  1: 0.2529442,
  2: 0.022839917,
  3: 0.67966133,
  4: 0.022271251,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0},
 {0: 0.033505727,
  1: 0.03383899,
  2: 0.033726145,
  3: 0.86513734,
  4: 0.033791844,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0},
 {0: 0.050436713,
  1: 0.050077148,
  2: 0.23769067,
  3: 0.61115456,
  4: 0.05064096,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0},
 {0: 0.040014304,
  1: 0.040304594,
  2: 0.040139057,
  3: 0.8392811,
  4: 0.04026095,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0},
 {0: 0.040151738,
  1: 0.04007229,
  2: 0.04002432,
  3: 0.83964187,
  4: 0.04010981,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0},
 {0: 0,
  1: 0,
  2: 0.1303221,
  3: 0.7679275,
  4: 0.082211085,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling